In [1]:
colab = False

if colab == True:
    !pip install nnunetv2
    !pip install torchio
    

In [2]:
import os
import torch
from torch.utils.data import random_split
import torchio as tio
from pathlib import Path
import importlib
import shutil
from collections import OrderedDict
import json
import numpy as np
import nnunetv2

if colab:
    from google.colab import drive
    drive.flush_and_unmount()
    drive.mount('/content/drive', force_remount=True)
    base_dir = '/content/drive/My Drive/Colab Notebooks/tumor-segmentation'
    os.chdir(base_dir)
    torch.cuda.empty_cache()

    !git clone https://github.com/MIC-DKFZ/nnUNet.git
    !git clone https://github.com/NVIDIA/apex

    training_size = 400


else:
    base_dir = '/home/asp/Downloads/DMIAI/DMIAI_2023/tumor-segmentation'
    training_size = 200

from utils import validate_segmentation, plot_prediction, dice_score
import torchio_utils
importlib.reload(torchio_utils)
from torchio_utils import torchio_compose_train, plot_example

path_to_mask = 'data/patients/labels/'
path_to_imgs = 'data/patients/imgs/'
path_to_controls = 'data/controls/imgs/' #healthy individuals (no tumors)
dataset_path = 'data.zip'
dataset_dir_name = 'data'
dataset_dir = Path(dataset_dir_name)

if not dataset_dir.is_dir():
    !curl --silent --output {dataset_path} --location {dataset_url}
    !unzip -qq {dataset_path}

In [3]:
def make_if_dont_exist(folder_path,overwrite=False):
    """
    creates a folder if it does not exists
    input: 
    folder_path : relative path of the folder which needs to be created
    over_write :(default: False) if True overwrite the existing folder 
    """
    if os.path.exists(folder_path):
        
        if not overwrite:
            print(f'{folder_path} exists.')
        else:
            print(f"{folder_path} overwritten")
            shutil.rmtree(folder_path)
            os.makedirs(folder_path)

    else:
      os.makedirs(folder_path)
      print(f"{folder_path} created!")

In [4]:
task_name = 'MIPPET'
nnunet_dir = 'nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data'
task_folder_name = os.path.join(nnunet_dir,task_name)
train_image_dir = os.path.join(task_folder_name,'imagesTr')
train_label_dir = os.path.join(task_folder_name,'labelsTr')
test_dir = os.path.join(task_folder_name,'imagesTs')
main_dir = os.path.join(base_dir,'nnUNet/nnunet')


make_if_dont_exist(task_folder_name,overwrite = False)
make_if_dont_exist(train_image_dir)
make_if_dont_exist(train_label_dir)
make_if_dont_exist(test_dir,overwrite= False)
make_if_dont_exist(os.path.join(main_dir,'nnunet_trained_models'))

nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/MIPPET exists.
nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/MIPPET/imagesTr exists.
nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/MIPPET/labelsTr exists.
nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/MIPPET/imagesTs exists.
/home/asp/Downloads/DMIAI/DMIAI_2023/tumor-segmentation/nnUNet/nnunet/nnunet_trained_models exists.


In [5]:
images_dir = dataset_dir / 'patients/imgs'
labels_dir = dataset_dir / 'patients/labels'
controls_dir = dataset_dir / 'controls/imgs'
image_paths = sorted(images_dir.glob('*.png'))
label_paths = sorted(labels_dir.glob('*.png'))
control_paths = sorted(controls_dir.glob('*.png'))

dataset_ID = 1

dataset = torchio_compose_train(image_paths, label_paths, control_paths, 
                                cropsize=(400,991), train_size = training_size,
                                dataset_ID=dataset_ID)

Found 182 subjects


  0%|          | 0/181 [00:00<?, ?it/s]

Landmarks: [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 4.33322500e-02 6.22397969e-01 9.23236779e+00
 1.72545437e+01 2.07381415e+01 2.53002322e+01 4.03121636e+01
 1.00000000e+02]
Augmenting 18 subjects


 22%|██▏       | 4/18 [00:01<00:06,  2.21it/s]/home/asp/anaconda3/lib/python3.11/site-packages/torchio/transforms/transform.py:163: RuntimeWarning: Input image is 2D, but "2" is in axes: (0, 1, 2)
  transformed = self.apply_transform(subject)
 89%|████████▉ | 16/18 [00:08<00:01,  1.62it/s]

In [ ]:
# Define the base path for your nnUNet directories
base_path = '/home/asp/Downloads/DMIAI/DMIAI_2023/tumor-segmentation'

# Set the environment variables
os.environ['nnUNet_raw'] = os.path.join(base_path, 'nnUNet_raw')
os.environ['nnUNet_preprocessed'] = os.path.join(base_path, 'nnUNet_preprocessed')
os.environ['nnUNet_results'] = os.path.join(base_path, 'nnUNet_results')

In [ ]:
!nnUNetv2_plan_and_preprocess -h

usage: nnUNetv2_plan_and_preprocess [-h] [-d D [D ...]] [-fpe FPE]
                                    [-npfp NPFP] [--verify_dataset_integrity]
                                    [--no_pp] [--clean] [-pl PL]
                                    [-gpu_memory_target GPU_MEMORY_TARGET]
                                    [-preprocessor_name PREPROCESSOR_NAME]
                                    [-overwrite_target_spacing OVERWRITE_TARGET_SPACING [OVERWRITE_TARGET_SPACING ...]]
                                    [-overwrite_plans_name OVERWRITE_PLANS_NAME]
                                    [-c C [C ...]] [-np NP [NP ...]]
                                    [--verbose]

options:
  -h, --help            show this help message and exit
  -d D [D ...]          [REQUIRED] List of dataset IDs. Example: 2 4 5. This
                        will run fingerprint extraction, experiment planning
                        and preprocessing for these datasets. Can of course
                        al

In [ ]:
!nnUNetv2_plan_and_preprocess -d 001 --verify_dataset_integrity


Fingerprint extraction...
Dataset001_MyDataset
Traceback (most recent call last):
  File "/home/asp/anaconda3/bin/nnUNetv2_plan_and_preprocess", line 8, in <module>
    sys.exit(plan_and_preprocess_entry())
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/asp/anaconda3/lib/python3.11/site-packages/nnunetv2/experiment_planning/plan_and_preprocess_entrypoints.py", line 182, in plan_and_preprocess_entry
    extract_fingerprints(args.d, args.fpe, args.npfp, args.verify_dataset_integrity, args.clean, args.verbose)
  File "/home/asp/anaconda3/lib/python3.11/site-packages/nnunetv2/experiment_planning/plan_and_preprocess_api.py", line 47, in extract_fingerprints
    extract_fingerprint_dataset(d, fingerprint_extractor_class, num_processes, check_dataset_integrity, clean,
  File "/home/asp/anaconda3/lib/python3.11/site-packages/nnunetv2/experiment_planning/plan_and_preprocess_api.py", line 30, in extract_fingerprint_dataset
    verify_dataset_integrity(join(nnUNet_raw, dataset_name), num_